In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

C:\Users\admin\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


*** Load the datasets of both Import and data bank ***

In [2]:
data_bank_df = pd.read_excel("D:/Fuzzy Cleaning/data bank.xlsx")   # load and read the Data_bank
data_bank_df.head()

,Product Description,Model Name,Brand Name
0,"1ST LENS ASSY (9147),A-2182-241-A,SEL1635GM(PA...",SEL1635GM,Sony
1,92992523 VPLL-Z4015/4WW SHORT FOCUS ZOOM LENS ...,VPLL-Z4015/4WW,Sony
2,16536659 FUJINON LENS GF32-64MMF4 R LM WR,GF32-64 mm,Fujifilm
3,16443060 F XF50-140MMF2.8 R LM OIS WR (LENS),XF50-140 mm,Fujifilm
4,16536659 FUJINON LENS GF32-64MMF4 R LM WR,GF32-64 mm,Fujifilm


In [3]:
import_df = pd.read_excel("D:/Fuzzy Cleaning/import.xlsx")   # Load and read Import data
import_df.head()

,slno,Date,HS Code,Product Description,Consignee Name,Shipper Name,Consignee Address 1,Consignee Address 2,Shipper Address1,Shipper Address 2,...,Raw Consignee Add1,Raw Consignee Add2,Raw Shipper Address1,Raw Shipper Address2,Raw Consignee City,Raw Consignee Pincode,Raw Consignee State,Raw Consignee Phone,Raw Consignee E-mail,Record Id
0,0,2023-06-01,85258900,M320 IVCIII CAMERA - 4K 10449087 SPARE PARTS ...,DHR HOLDING INDIA PRIVATE LIMITED,LEICA MIKROSYSTEME VERTRIEB GMBH -DSA,"UNIT NO 325 TO 328 DLF TOWERS,SHIV AJI MARG",NaN,ERNST-LEITZ-STRASSE 17-3735578 WETZLAR / DUETS...,NaN,...,"UNIT NO 325 TO 328 DLF TOWERS,SHIV AJI MARG",NaN,ERNST-LEITZ-STRASSE 17-3735578 WETZLAR,NaN,DELHI,110015.0,DELHI,NaN,amit.jain@danaher.com,41162905
1,1,2023-06-01,85258900,16759732 FUJIFILM DIGITAL CAMERA F X-T30 II B/...,FUJIFILM INDIA PRIVATE LIMITED,FUJIFILM CORPORATION,"BUSINESS CENTRE OFF NO 521 LEVEL V ,CADDIE COM...",NaN,"26-30 NASHIAZABU,2 CHOME MINATO -KUTOKYO JAPAN...",NaN,...,"BUSINESS CENTRE OFF NO 521 LEVEL V ,CADDIE COM...",NaN,"26-30 NASHIAZABU,2 CHOME MINATO -KUTOKYO JAPAN...",NaN,NEW DELHI,110037.0,DELHI,NaN,amit.kumar@fujifilm.com,40910267
2,2,2023-06-01,85258900,16670041 FUJIFILM DIGITAL CAMERA F X-S10 CD LI...,FUJIFILM INDIA PRIVATE LIMITED,FUJIFILM CORPORATION,"BUSINESS CENTRE OFF NO 521 LEVEL V ,CADDIE COM...",NaN,"26-30 NASHIAZABU,2 CHOME MINATO -KUTOKYO JAPAN...",NaN,...,"BUSINESS CENTRE OFF NO 521 LEVEL V ,CADDIE COM...",NaN,"26-30 NASHIAZABU,2 CHOME MINATO -KUTOKYO JAPAN...",NaN,NEW DELHI,110037.0,DELHI,NaN,amit.kumar@fujifilm.com,40910266
3,3,2023-06-01,85258900,M320 IVCIII CAMERA - 4K 10449087 SPARE PARTS ...,DHR HOLDING INDIA PRIVATE LIMITED,LEICA MIKROSYSTEME VERTRIEB GMBH -DSA,"UNIT NO 325 TO 328 DLF TOWERS,SHIV AJI MARG",NaN,ERNST-LEITZ-STRASSE 17-3735578 WETZLAR / DUETS...,NaN,...,"UNIT NO 325 TO 328 DLF TOWERS,SHIV AJI MARG",NaN,ERNST-LEITZ-STRASSE 17-3735578 WETZLAR,NaN,DELHI,110015.0,DELHI,NaN,amit.jain@danaher.com,41162884
4,4,2023-06-01,85258900,16670041 FUJIFILM DIGITAL CAMERA F X-S10 CD LI...,FUJIFILM INDIA PRIVATE LIMITED,FUJIFILM CORPORATION,"BUSINESS CENTRE OFF NO 521 LEVEL V ,CADDIE COM...",NaN,"26-30, NISHIAZABU 2-CHOME MINATO-KUTOKYO 106-8...",NaN,...,"BUSINESS CENTRE OFF NO 521 LEVEL V ,CADDIE COM...",NaN,"26-30, NISHIAZABU 2-CHOME MINATO-KUTOKYO 106-8...",NaN,NEW DELHI,110037.0,DELHI,NaN,amit.kumar@fujifilm.com,41654246


In [4]:
# Data cleaning: Drop rows with missing values in the 'Product Description' column
data_bank_df.dropna(subset=['Product Description'], inplace=True)
import_df.dropna(subset=['Product Description'], inplace=True)

In [5]:
# To check if there is still any missing values present there

data_bank_df['Product Description'].isnull().sum()

0

In [6]:
# to check in import_df 
import_df['Product Description'].isnull().sum()

0

*** As we find out there is no missing values present in our dataset now we do further tasks ***

In [23]:
exact_matches = pd.merge(import_df, data_bank_df, how='inner', on='Product Description')
result_task1 = exact_matches[['Brand Name_y', 'Model Name_y']]
result_task1.columns = ['Brand Name', 'Model Name']

*** Task 2 ***

In [8]:
import swifter

# Task 2 - Find matches with up to 90% similarity between product descriptions.
def fuzzy_match(row):
    choices = data_bank_df['Product Description'].tolist()
    match = process.extractOne(row['Product Description'], choices, scorer=fuzz.token_sort_ratio)
    if match[1] >= 90:
        return data_bank_df[data_bank_df['Product Description'] == match[0]][['Brand Name', 'Model Name']].values[0]
    else:
        return None

import_df['Brand Name'] = None
import_df['Model Name'] = None
task2_matches = import_df.swifter.apply(fuzzy_match, axis=1, result_type='expand')
import_df['Brand Name'] = task2_matches[0]
import_df['Model Name'] = task2_matches[1]
result_task2 = import_df.dropna(subset=['Brand Name', 'Model Name'])
result_task2 = result_task2[['Brand Name', 'Model Name']]



Pandas Apply:   0%|          | 0/1547 [00:00<?, ?it/s]

In [20]:
# Assuming you have a DataFrame named 'brand_table' containing the Brand Name and additional information.
brand_table = pd.read_csv('D:/Fuzzy Cleaning/brand.csv')

# Preprocess the data
import_df['Shipper Name'] = import_df['Shipper Name'].str.lower().str.replace('[^\w\s]', '').str.strip()
brand_table['Brand Name'] = brand_table['Brand Name'].str.lower().str.replace('[^\w\s]', '').str.strip()

# Create a dictionary for fast lookup of Brand Name in the brand_table DataFrame
brand_name_lookup = brand_table.set_index('Brand Name')['slno'].to_dict()

def find_brand(row):
    shipper_name = row['Shipper Name']
    match = process.extractOne(shipper_name, brand_name_lookup.keys(), scorer=fuzz.token_sort_ratio)
    if match[1] >= 90:
        return brand_name_lookup[match[0]]
    else:
        return None

import_df['Brand Name'] = import_df.apply(find_brand, axis=1)
result_task3 = import_df.dropna(subset=['Brand Name', 'Model Name'])
result_task3 = result_task3[['Brand Name', 'Model Name']]


*** Now combine all and export and save into csv or excel format ***

In [24]:
final_result = pd.concat([result_task1, result_task2, result_task3]).reset_index(drop=True)
final_result.to_csv('final_result.csv', index=False)